In [2]:
from pathlib import Path
import tarfile

import pandas as pd

import fastai.basics

from utils.prepare_data import target_columns, read_data, add_regime_kind_column
from utils.project_parameters import neural_network_batch_size
import utils.deeplearning

In [3]:
target_col = target_columns[snakemake.wildcards["target"]]

if snakemake.params["application_type"] == "testing":
    df = read_data(snakemake.input["sim_parameters"])
    if target_col == "regime_kind":
        df = add_regime_kind_column(df)
elif snakemake.params["application_type"] == "empirical": 
    with tarfile.open(snakemake.input["data"]) as t:
        names = [name.replace(".npy", "") for name in t.getnames() if name.endswith(".npy")]
    df = pd.DataFrame({"uuid": names, target_col: None})  
    
dataset = utils.deeplearning.SweepsDataset(
    snakemake.input["data"], df, None
)

In [4]:
with open(snakemake.input["model_labels"]) as f:
    labels = [line.strip() for line in f]

In [5]:
model = fastai.basics.load_learner(fname=str(Path(snakemake.input["model_object"])))
model = model.load(str(Path(snakemake.input["fit_model"]).stem))

In [6]:
results = utils.deeplearning.get_application_inferences(model, dataset, neural_network_batch_size, labels, target_col)
results.to_csv(snakemake.output["inferences"], index=True, sep='\t')